# AOC 2025 Day 8

In [1]:
@file:DependsOn("com.toldoven.aoc:aoc-kotlin-notebook:1.1.2")

In [2]:
import com.toldoven.aoc.notebook.AocClient

val aocClient = AocClient.fromEnv().interactiveDay(2025, 8)
val input = aocClient.input()
aocClient.viewPartOne()

--- Day 8: Playground --- 
 Equipped with a new understanding of teleporter maintenance, you confidently step onto the repaired teleporter pad. 
 You rematerialize on an unfamiliar teleporter pad and find yourself in a vast underground space which contains a giant playground! 
 Across the playground, a group of Elves are working on setting up an ambitious Christmas decoration project. Through careful rigging, they have suspended a large number of small electrical junction boxes . 
 Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes. 
 The Elves are trying to figure out which junction boxes to connect so that electricity can reach every junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input). 
 For example: 
 162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
 
 This list describes the position of 20 junction boxes, one per line. Each position is given as X,Y,Z coordinates. So, the first junction box in the list is at X=162 , Y=817 , Z=812 . 
 To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are as close together as possible according to straight-line distance . In this example, the two junction boxes which are closest together are 162,817,812 and 425,690,689 . 
 By connecting these two junction boxes together, because electricity can flow between them, they become part of the same circuit . After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits. 
 Now, the two junction boxes which are closest together but aren't already directly connected are 162,817,812 and 431,825,988 . After connecting them, since 162,817,812 is already connected to another junction box, there is now a single circuit which contains three junction boxes and an additional 17 circuits which contain one junction box each. 
 The next two junction boxes to connect are 906,360,560 and 805,96,715 . After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each. 
 The next two junction boxes are 431,825,988 and 425,690,689 . Because these two junction boxes were already in the same circuit , nothing happens! 
 This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be. 
 After making the ten shortest connections, there are 11 circuits: one circuit which contains 5 junction boxes, one circuit which contains 4 junction boxes, two circuits which contain 2 junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces 40 . 
 Your list contains many junction boxes; connect together the 1000 pairs of junction boxes which are closest together. Afterward, what do you get if you multiply together the sizes of the three largest circuits?

In [4]:
data class JunctionBox(
    val x: Int,
    val y: Int,
    val z: Int
) {

    /**
     * determine distance between two junction boxes
     */
    fun getDistance(other: JunctionBox): Double {
        // Implementation of Euclidean distance formula: d = sqrt((x2-x1)^2 + (y2-y1)^2 + (z2-z1)^2)
        val dx = (x - other.x).toDouble()
        val dy = (y - other.y).toDouble()
        val dz = (z - other.z).toDouble()

        return sqrt(dx.pow(2) + dy.pow(2) + dz.pow(2))
    }
}

class UnionFind(size: Int) {
    private val parent = IntArray(size) { it }
    private val rank = IntArray(size) { 1 }

    fun find(x: Int): Int {
        if (parent[x] != x) {
            parent[x] = find(parent[x])
        }
        return parent[x]
    }

    fun union(x: Int, y: Int): Boolean {
        val rootX = find(x)
        val rootY = find(y)

        if (rootX == rootY) return false // already in same circuit

        // union by rank
        when {
            rank[rootX] < rank[rootY] -> parent[rootX] = rootY
            rank[rootX] > rank[rootY] -> parent[rootY] = rootX
            else -> {
                parent[rootY] = rootX
                rank[rootX]++
            }
        }
        return true
    }

    fun getCircuitSizes(): List<Int> {
        return parent.indices
            .groupBy { find(it) }
            .values
            .map { it.size }
    }
}

val lines = input.split("\n")
val boxes = lines.map {
    val (x, y, z) = it.split(",").map { it.trim().toInt() }
    JunctionBox(x, y, z)
}

// all pairs with distances sorted by distance
val distancePairs = boxes.indices.flatMap { i ->
    (i + 1 until boxes.size).map { j ->
        Triple(i, j, boxes[i].getDistance(boxes[j]))
    }
}.sortedBy { it.third }

val uf = UnionFind(boxes.size)

for ((index, pair) in distancePairs.withIndex()) {
    if (index >= 1000) break
    val (i, j, _) = pair
    uf.union(i, j)
}

val largestTree = uf.getCircuitSizes()
    .sortedDescending()
    .take(3)

val answerPartOne = largestTree
    .map { it.toLong() }
    .reduce { acc: Long, size -> acc * size }

aocClient.submitPartOne(answerPartOne)

Your answer: 24360. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [5]:
aocClient.viewPartTwo()

--- Part Two --- 
 The Elves were right; they definitely don't have enough extension cables. You'll need to keep connecting junction boxes together until they're all in one large circuit . 
 Continuing the above example, the first connection which causes all of the junction boxes to form a single circuit is between the junction boxes at 216,146,977 and 117,168,530 . The Elves need to know how far those junction boxes are from the wall so they can pick the right extension cable; multiplying the X coordinates of those two junction boxes ( 216 and 117 ) produces 25272 . 
 Continue connecting the closest unconnected pairs of junction boxes together until they're all in the same circuit . What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?

In [7]:
val uf = UnionFind(boxes.size)
var lastConnectionIndizes: Pair<Int, Int>? = null

var successfulUnions = 0

for ((i, j, _) in distancePairs) {
    if (uf.union(i, j)) {
        lastConnectionIndizes = i to j
        successfulUnions++
        if (successfulUnions == boxes.size - 1) break
    }
}

val (idxA, idxB) = lastConnectionIndizes!!
val answerPartTwo = boxes[idxA].x.toLong() * boxes[idxB].x.toLong()

aocClient.submitPartTwo(answerPartTwo)

Your answer: 2185817796. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 8! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .